In [1]:
#!/usr/bin/env python
# coding: utf-8

# In[1]:


from __future__ import print_function, division
import os

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="2,3"

from itertools import chain
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from IPython.core.debugger import set_trace
import itertools
import seaborn as sns
from tqdm import tqdm
import random
import cv2
from natsort import natsorted
import collections
import skimage
from IPython import display
import pylab as pl
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics.regression import mean_absolute_error, mean_squared_error, r2_score, explained_variance_score
import numpy as np
from torch import nn
import torch.nn.functional as F
import torchvision.transforms.functional as TF


from skorch import NeuralNetRegressor
from skorch.helper import predefined_split
from sklearn.metrics.regression import mean_absolute_error, mean_squared_error, r2_score, explained_variance_score
from skorch import callbacks

from dask.distributed import Client
from sklearn.externals import joblib
from sklearn.model_selection import GridSearchCV


plt.ion()
%matplotlib inline

# # Check cuda.is_available ?

# In[2]:


cuda_available = torch.cuda.is_available()
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
print("cuda_available : {}, device : {}".format(cuda_available, device))


# # Define Dataset & DataLoader

# In[3]:

MAXLEN = 300
FPS = 24.0

def drop_huge_seq(input_df, save_path="./preprocess/example_data/person_detection_and_tracking_results_drop.pkl"):
    if os.path.exists(save_path):
        print('Already dropped! Return...')
        return
    
    vids = list(set(input_df.vids))

    for i in tqdm(range(len(vids)), desc='DropInputSeq '):
        slice_df = input_df.loc[input_df.vids==vids[i]]
        if slice_df.values.shape[0] > MAXLEN:
            input_df.iloc[slice_df.index] = np.nan * np.ones_like(slice_df.values)

    # drop Nans !
    res_df = input_df.dropna()
    res_df.to_pickle("./preprocess/example_data/person_detection_and_tracking_results_drop.pkl")

input_df = pd.read_csv('./preprocess/example_data/person_detection_and_tracking_results.csv',
                       sep='\t', names=['vids', 'idx', 'pos'])
    
# drop huge seq
drop_huge_seq(input_df, save_path="./preprocess/example_data/person_detection_and_tracking_results_drop.pkl")


# In[4]:


df = pd.read_pickle('./preprocess/example_data/person_detection_and_tracking_results_drop.pkl')
len(list(set(df.vids)))


# In[5]:


df = pd.read_pickle("./preprocess/data/targets_dataframe.pkl")
target_columns = df.columns.values[:-2]
# target_columns = ['Toe In / Out/L', 'Toe In / Out/R']


# In[6]:


def pid2vid(pid):
    num, test_id, trial_id = pid.split('_')
    return '_'.join([num, 'test', test_id, 'trial', trial_id])
    

def vid2pid(vid):
    split = vid.split('_')
    return '_'.join([split[0], split[2], split[4]])



class GAITDataset(Dataset):
    def __init__(self,
                 X, y, scaler, frame_home="/data/GaitData/CroppedFrameArrays", maxlen=300):
        
        self.frame_home = frame_home
        
        self.X = X
        self.y = y
        self.vids = [ pid2vid(pid) for pid in self.y.index ]
        
        self.maxlen = maxlen
        
        if scaler:
            scaled_values = scaler.transform(y)
            self.y.loc[:,:] = scaled_values
            
    def __len__(self):
        return len(self.vids)
    
    def __getitem__(self, idx):
        
        vid = self.vids[idx]
        positions = [ eval(val) for val in self.X.loc[self.X.vids==vid].pos.values ]
        
        
        stacked_arr = np.load(os.path.join(self.frame_home, vid) + '.npy')
        
        inputs = []        

        for cropped in stacked_arr:  
            pic = cv2.resize(cv2.cvtColor(cropped, cv2.COLOR_BGR2GRAY), (64,64))[:,:,None]
            
            pic = TF.to_tensor(pic) # scale to [0.0, 1.0]
            pic = TF.normalize(pic, (0.5,), (0.5,)).permute(1,2,0).numpy()   # scale to [-1.0, 1.0]
            inputs.append(pic)
            
        targets = self.y.loc[vid2pid(vid)].values

        # zero padding
        inputs = np.pad(inputs, ((0,self.maxlen-len(inputs)),(0,0),(0,0),(0,0)),
                                               'constant', constant_values=0).transpose(3,0,1,2)
        
        return torch.tensor(inputs, dtype=torch.float32), torch.tensor(inputs, dtype=torch.float32)

/home/hossay/anaconda3/envs/torch/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


cuda_available : True, device : cuda:0
Already dropped! Return...


In [2]:
class Conv3d_with_same_padding(nn.Conv3d):
    def __init__(self, in_channels,
                       out_channels,
                       kernel_size,
                       stride=1,
                       padding=0,
                       dilation=1,
                       groups=1,
                       bias=True,
                       padding_type='same'):
        
        super(Conv3d_with_same_padding, self).__init__(in_channels,
                                     out_channels,
                                     kernel_size,
                                     stride,
                                     padding,
                                     dilation,
                                     groups,
                                     bias)
        
        self.padding_type = padding_type
    
    def forward(self, x, debug=False):
        n,c,d,h,w = x.size()
        if self.padding_type == 'same':
            padding_need = []
            for i,e in enumerate([d,h,w]):
                bias = 0.5 if self.stride[i] % 2 == 0 else 0.0
                padding_need.append(round((e * (self.stride[i]-1) + self.kernel_size[i] - self.stride[i]) / 2 + bias))
            
            padding_need = tuple(padding_need)
            
        if debug:
            set_trace()

        return F.conv3d(x, self.weight, self.bias, self.stride, 
                        padding_need, self.dilation, self.groups)


# In[ ]:


class ResidualBlock(nn.Module):
    def __init__(self, C_in, C_out, pool, highway=True):
        super(ResidualBlock, self).__init__()
        self.pool = pool
        self.highway = highway
                
        stride = 1
        
        if C_in != C_out:
            C = C_out
        else:
            C = C_in = C_out
            
        if pool:
            # input dimension matchig
            self.conv_matching = Conv3d_with_same_padding(C_in, C, kernel_size=1, stride=1, padding_type='same')
            self.bn_matching = nn.BatchNorm3d(C)

            # for pooling of residual path
            stride = 2
            self.conv_pool = Conv3d_with_same_padding(C_in, C, kernel_size=1, stride=2, padding_type='same')
            self.bn_pool= nn.BatchNorm3d(C)
                
        # conv_a : reduce number of channels by factor of 4 (output_channel = C/4)
        self.conv_a = Conv3d_with_same_padding(C, int(C/4), kernel_size=1, stride=stride, padding_type='same')
        self.bn_a = nn.BatchNorm3d(int(C/4))
        
        # conv_b : more wide receptive field (output_channel = C/4)
        self.conv_b = Conv3d_with_same_padding(int(C/4), int(C/4), kernel_size=3, stride=1, padding_type='same')
        self.bn_b = nn.BatchNorm3d(int(C/4))
        
        # conv_c : recover org channel C (output_channel = C)
        self.conv_c = Conv3d_with_same_padding(int(C/4), C, kernel_size=1, stride=1, padding_type='same')
        self.bn_c = nn.BatchNorm3d(C)
        
        if highway:
            # conv_g : gating for highway network
            self.conv_g = Conv3d_with_same_padding(C, C, kernel_size=1, stride=1, padding_type='same')
        
    
    def forward(self, x):
        '''
            x : size = (batch, channels, maxlen, height, width)
        '''
        
        res = x
        
        if self.pool:
            # input dimension matching with 1x1 conv
            x = self.conv_matching(x)
            x = self.bn_matching(x)
            
            # pooling of residual path
            res = self.conv_pool(res)
            res = self.bn_pool(res)
        
        # conv_a (C/4)
        x = self.conv_a(x)
        x = self.bn_a(x)
        x = F.relu(x)
        
        # conv_b (C/4)
        x = self.conv_b(x)
        x = self.bn_b(x)
        x = F.relu(x)
        
        # conv_c (C)
        x = self.conv_c(x)
        x = self.bn_c(x)
        
        if self.highway:
            # gating mechanism from "highway network"
            
            # gating factors controll intensity between x and f(x)
            # gating = 1.0 (short circuit) --> output is identity (same as initial input)
            # gating = 0.0 (open circuit)--> output is f(x) (case of non-residual network)
            gating = torch.sigmoid(self.conv_g(x))
            
            # apply gating mechanism
            x = gating * res + (1.0 - gating) * F.relu(x)

            
        else:
            # normal residual ops (addition)
            x = F.relu(x) + res
        
        return x


# In[ ]:


class View(nn.Module):
    def __init__(self, *shape):
        super(View, self).__init__()
        self.shape = shape
    def forward(self, x):
        return x.view(*self.shape)

class GAP(nn.Module):
    def __init__(self):
        super(GAP, self).__init__()
    def forward(self, x):
        '''
        
            x : size = (N,C,D,H,W)
        '''
        set_trace()
        return torch.mean(x, (2,3,4))

class HighWay(nn.Module):
    def __init__(self, input_channel=1, 
                 num_layers = [3,4,6], num_filters = [64,128,256]):
        
        super(HighWay, self).__init__()
        
        self.num_layers = num_layers
        self.num_filters = num_filters

        def res_blocks(residual_blocks, num_layers, num_filters, block_ix, pool_first_layer=True):
            block_layers = num_layers[block_ix]

            for i in range(block_layers):
                # default values
                pool = False
                block_filters = num_filters[block_ix]
                
                C_in = C_out = block_filters
                
                if pool_first_layer and i==0:
                    pool = True
                if i==0 and block_ix > 0:
                    C_in = num_filters[block_ix-1]
                    
                print(f"layer : {i}, block : {block_ix}, C_in/C_out : {C_in}/{C_out}")
                residual_blocks.append(ResidualBlock(C_in=C_in, C_out=C_out,pool=pool, highway=True))
                
        residual_blocks = []

        for i in range(len(num_layers)):
            pool_first_layer = True
            if i == 0:
                pool_first_layer = False
            res_blocks(residual_blocks, num_layers=num_layers, num_filters=num_filters, block_ix=i,
                       pool_first_layer=pool_first_layer)
        
        
        nn.ReLU
        self.model = nn.Sequential(nn.Conv3d(input_channel, num_filters[0], kernel_size=(3,7,7), stride=(1,1,1)),
                                   nn.BatchNorm3d(num_filters[0]), 
                                   nn.ReLU(),
                                   nn.Conv3d(num_filters[0], num_filters[1], kernel_size=3, stride=2),
                                   nn.BatchNorm3d(num_filters[1]), 
                                   nn.ReLU(),
                                   nn.MaxPool3d(kernel_size=(3,), stride=2),
                                   nn.Conv3d(num_filters[1], num_filters[2], kernel_size=3, stride=2),
                                   nn.BatchNorm3d(num_filters[2]), 
                                   nn.ReLU(),
                                   #*residual_blocks,
                                   GAP(),
                                   nn.Linear(num_filters[-1], 17)
                                   )

    def forward(self, img):
        '''
            img : size = (batch, input_channel, maxlen, height, width)
        '''

        return self.model(img)

    
class Concat(nn.Module):
    def __init__(self, val, dim):
        super(Concat, self).__init__()
        
        self.val = val
        self.dim = dim
        
    def forward(self, x):
        return torch.cat([self.val, x], self.dim)

    
class Encoder(nn.Module):
    def __init__(self, num_filters = [1,32,64,128,256]):
        
        super(Encoder, self).__init__()
        
        self.encode = nn.Sequential(
            # b, 32, 150, 32, 32
            nn.Conv3d(num_filters[0], num_filters[1], kernel_size=4, stride=2, padding=1), # ()
            nn.BatchNorm3d(num_filters[1]), 
            nn.ReLU(True),
            
            # b, 64, 75, 16, 16
            nn.Conv3d(num_filters[1], num_filters[2], kernel_size=4, stride=2, padding=1),
            nn.BatchNorm3d(num_filters[2]), 
            nn.ReLU(True),
            
            # b, 128, 25, 8, 8
            nn.Conv3d(num_filters[2], num_filters[3], kernel_size=(5,4,4), stride=(3,2,2), padding=1),
            nn.BatchNorm3d(num_filters[3]), 
            nn.ReLU(True),
            
            # b, 256, 8, 4, 4
            nn.Conv3d(num_filters[3], num_filters[4], kernel_size=(6,4,4), stride=(3,2,2), padding=1),
            nn.BatchNorm3d(num_filters[4]), 
            nn.ReLU(True),            
            
         )

    def forward(self, x):
        '''
            x : size = (B, C, D, H, W)
        '''
        return self.encode(x)

class Decoder(nn.Module):
    def __init__(self, enc_feats, num_filters = [256,128,64,32,1]):
        
        super(Decoder, self).__init__()
        
        self.decode = nn.Sequential(
            # b, 128, 25, 8, 8
            nn.ConvTranspose3d(num_filters[0], num_filters[1], kernel_size=(6,4,4), stride=(3,2,2), padding=1),
            nn.BatchNorm3d(num_filters[1]), 
            nn.ReLU(True),
            
            Concat(enc_feats[-2], 1),
            
            # b, 64, 75, 16, 16
            nn.ConvTranspose3d(2*num_filters[1], num_filters[2], kernel_size=(5,4,4), stride=(3,2,2), padding=1),
            nn.BatchNorm3d(num_filters[2]), 
            nn.ReLU(True),
            
            Concat(enc_feats[-3], 1),

            # b, 32, 150, 32, 32
            nn.ConvTranspose3d(2*num_filters[2], num_filters[3], kernel_size=4, stride=2, padding=1),
            nn.BatchNorm3d(num_filters[3]), 
            nn.ReLU(True),
            
            
            Concat(enc_feats[-4], 1),
            
            # b, 1, 300, 64, 64
            nn.ConvTranspose3d(2*num_filters[3], num_filters[4], kernel_size=4, stride=2, padding=1),
            nn.Tanh()            
            
        )

    def forward(self, x):
        '''
            x : size = (B, C, D, H, W)
        '''
        return self.decode(x)
    

    
    
class AutoEncoder(nn.Module):
    def __init__(self):
        super(AutoEncoder, self).__init__()
        
        self.encoder = Encoder()
    
    def encode(self, X):
        enc_feats = []
        for layer in self.encoder.encode:
            if type(layer).__name__ == 'ReLU':
                enc_feats.append(X)
            X = layer(X)
        
        encoded = enc_feats[-1]
        
        return Decoder(enc_feats).to(device), encoded
    
            
    def forward(self, X):
        decoder, encoded = self.encode(X)
        decoded = decoder(encoded)
        return decoded, encoded  # <- return a tuple of two values
    
    
class AutoEncoderNet(NeuralNetRegressor):
    def get_loss(self, y_pred, y_true, *args, **kwargs):
        decoded, encoded = y_pred  # <- unpack the tuple that was returned by `forward`
        loss_reconstruction = super(AutoEncoderNet, self).get_loss(decoded, y_true, *args, **kwargs)
        loss_l1 = 1e-3 * torch.abs(encoded).sum()
        
        return loss_reconstruction + loss_l1        

In [3]:
from sklearn.model_selection import train_test_split

def filter_input_df_with_vids(df, vids):
    return df[df['vids'].isin(vids)]

def filter_target_df_with_vids(df, vids):
    target_ids = [ vid2pid(vid) for vid in vids ]
    return df.loc[target_ids]

def split_dataset_with_vids(input_df, target_df, vids, test_size=0.3, random_state=42):
    train_vids, test_vids = train_test_split(vids, test_size=test_size, random_state=random_state)

    train_X, train_y = filter_input_df_with_vids(input_df,train_vids), filter_target_df_with_vids(target_df,train_vids)
    test_X, test_y = filter_input_df_with_vids(input_df,test_vids), filter_target_df_with_vids(target_df, test_vids)
        
    return train_X, train_y, train_vids, test_X, test_y, test_vids


def mape(y_true, y_pred):
    ape = []
    zero_cnt = 0
    for true,pred in zip(y_true, y_pred):
        if 0.0 not in true:
            ape.append(np.abs((true-pred)/true))  
        else:
            zero_cnt += 1
    
    return np.mean(ape), zero_cnt

In [4]:
from skorch.callbacks import Callback
from torchvision.utils import save_image

def to_img(x):
    x = 0.5 * (x + 1)
    x = np.clip(x, 0.0, 1.0)
    x = 255*x
    return x.astype(np.uint8)

def to_tensor_img(x):
    x = 0.5 * (x + 1)
    x = x.clamp(0, 1)
    x = x.view(x.size(0), 1, 64, 64)
    return x

class SaveResults(Callback):
    def __init__(self, path):
        self.path = path
        
    def on_epoch_end(self, net, **kwargs):        
        for name in ['train', 'valid']:
            dataset = kwargs['dataset_'+name]
            rand_ix = np.random.randint(len(dataset))
            X,y = dataset[rand_ix]
            
            save_dir = os.path.join(self.path, name)
            if not os.path.exists(save_dir):
                os.makedirs(save_dir)
            
            # target img
            y = y.numpy().transpose(1,2,3,0)  # (maxlen,h,w,3)
            
            # predicted img
            pred = net.predict(X[None,:])[0].transpose(1,2,3,0) # (maxlen,h,w,3)
            
            for sub_name,pic in zip(['target', 'pred'], [y,pred]):
                pic = to_tensor_img(torch.from_numpy(pic))
                save_image(pic, os.path.join(save_dir,sub_name+'.png'))

In [5]:


# dataset path
input_file = "./preprocess/example_data/person_detection_and_tracking_results_drop.pkl"
target_file = "./preprocess/data/targets_dataframe.pkl"

input_df = pd.read_pickle(input_file)
target_df = pd.read_pickle(target_file)[target_columns]

possible_vids = list(set(input_df.vids))[:100]
train_X, train_y, train_vids, test_X, test_y, test_vids = split_dataset_with_vids(input_df, target_df, possible_vids, test_size=0.3, random_state=42)

# target scaler
# scaler = StandardScaler()
# train_y.loc[:,:] = scaler.fit_transform(train_y.values)

scaler = None

# holdouf test set for final evaluation
test_dataset = GAITDataset(test_X, test_y, scaler)
test_batcher = DataLoader(test_dataset,batch_size=10, shuffle=False, num_workers=16)

from sklearn.model_selection import KFold
kf = KFold(n_splits=5)

train_vids = np.array(train_vids)


from torch.nn.modules.loss import _Loss

class MyCriterion(_Loss):
    def __init__(self):
        super(MyCriterion, self).__init__()
    
    def forward(self, x, y):
        valid_mask = ~(y.view(y.size(0),MAXLEN,-1)==0).all(dim=2)
        valid_mask = valid_mask.float()
        return torch.mean(torch.sum((valid_mask * ((x-y)**2).mean((1,3,4))),1)/torch.sum(valid_mask,1))
        
    
# for parallelism
#client = Client('127.0.0.1:8786')

# cross validation loop
scores = {'MAPE': [], 'MAE': [], 'RMSE': [], 'R2': [], 'Explained variation': []}

for train, valid in kf.split(train_vids):
    # split trainset with train/valid
    train_split, valid_split = train_vids[train], train_vids[valid]
    
    train_X, train_y = filter_input_df_with_vids(input_df,train_split), filter_target_df_with_vids(target_df,train_split)
    valid_X, valid_y = filter_input_df_with_vids(input_df,valid_split), filter_target_df_with_vids(target_df,valid_split)


    # dsataset !!
    train_dataset = GAITDataset(train_X, train_y, scaler)
    valid_dataset = GAITDataset(valid_X, valid_y, scaler)
        
    # Init net !
    net = AutoEncoderNet(
        AutoEncoder,
        batch_size=10,
        max_epochs=100,
        lr=1e-3,
        optimizer=torch.optim.Adam,
        #optimizer__weight_decay=1e-5,
        #optimizer__momentum=0.9,
        #optimizer__nesterov=True,
        criterion=MyCriterion,
        device='cuda',
        train_split=predefined_split(valid_dataset),
        # Shuffle training data on each epoch
        iterator_train__shuffle=True,
        callbacks=[#('ealy_stop', callbacks.EarlyStopping()),
                   #('lr_scheduler', callbacks.LRScheduler(policy='WarmRestartLR', base_period=2)),
                   ('prog_bar', callbacks.ProgressBar()),
                   ('save_results', SaveResults(path='./results'))
                   ],
    
    )
    
    #with joblib.parallel_backend('dask'):
    # fit with train set
    net.fit(train_dataset, y=None)

  epoch    train_loss    valid_loss     dur
-------  ------------  ------------  ------
      1       51.4635       79.4709  9.3011


      2       42.1256       42.9796  7.2749


      3       36.1266       40.9593  7.2143


      4       32.2057       28.1359  7.2062


      5       29.3680       24.8921  7.2725


...

     96        0.5238        0.4770  7.2574


     97        0.6233        0.6102  7.2462


     98        0.6102        0.4027  7.2518


     99        0.6024        0.6293  7.2698


    100        0.5858        0.5365  7.2128


In [6]:
print('MAE :', mean_absolute_error(y_true, y_pred, multioutput='raw_values'))
print('MSE :', mean_squared_error(y_true, y_pred, multioutput='raw_values'))
print('RMSE :', np.sqrt(mean_squared_error(y_true, y_pred, multioutput='raw_values')))
print('R^2 : ', r2_score(y_true, y_pred, multioutput='variance_weighted'))
print('Explained variation : ',explained_variance_score(y_true, y_pred, multioutput='variance_weighted'))

NameError: name 'y_true' is not defined

In [ ]:
y_pred = []
y_true = []

train_batcher = DataLoader(train_dataset,batch_size=10, shuffle=False, num_workers=16)

for b in iter(train_batcher):
    X_test, y_test = b
    y_pred.append(net.predict(X_test.numpy()))
    y_true.append(y_test.numpy())

y_pred = np.concatenate(y_pred, axis=0)
y_true = np.concatenate(y_true, axis=0)

if scaler:
    y_pred = scaler.inverse_transform(y_pred)
    y_true = scaler.inverse_transform(y_true)

In [ ]:
y_pred[0]

In [ ]:
y_true[0]

In [ ]:
for ii in range(17):
    xx = np.linspace(min(y_true[:,ii]), max(y_true[:,ii]))
    plt.scatter(y_pred[:,ii], y_true[:,ii])
    plt.plot(xx,xx)
    plt.show()

In [ ]:
y_true.mean(axis=0)

In [ ]:
print(scaler.inverse_transform(train_y.values).mean(axis=0))
print(scaler.inverse_transform(train_y.values).std(axis=0))

In [ ]:
print(scaler.inverse_transform(test_y.values).mean(axis=0))
print(scaler.inverse_transform(test_y.values).std(axis=0))

In [ ]:
from sklearn.datasets import fetch_olivetti_faces

In [ ]:
olivetti = fetch_olivetti_faces()

In [ ]:
olivetti['data'].shape

In [ ]:
olivetti['target'].shape

In [ ]:
from sklearn.datasets import load_linnerud
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.multioutput import MultiOutputRegressor

linnerud = load_linnerud()

X = linnerud.data
Y = linnerud.target


In [ ]:
Y

In [ ]:
model = MultiOutputRegressor(GradientBoostingRegressor(), n_jobs=-1)
model.fit(X, Y)

In [ ]:
model.predict(X)

In [ ]:
Y